# STT, ChatGPT, TTS Inference Pipeline TEST

In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer
from datasets import load_dataset

import librosa
import openai
openai.api_key = ##Secret Key##

import time
from gtts import gTTS
import IPython.display as ipd

In [2]:
# Load Models
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model_baseline = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small") # Parameters: 244M (1)
model_noising = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small") # 소음 데이터로 학습 (2)
model_denoising = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising") # DNS 통과 데이터로 학습
model_noising_denoising = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising_2") # 소음 데이터 + DNS 통과 데이터로 학습
model_noising_denoising_2 = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising_3") # 소음 데이터 + DNS 통과 데이터로 학습_2
model_noising_denoising_all = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising_all") # 소음 데이터 + DNS 통과 데이터로 학습 (4)
model_noising_denoising_all_pure = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising_all_pure") # DNS 통과 데이터로 학습 (3)
model_noising_2 = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-noising_2")
model_noising_3 = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-noising_3") # noise -> denoise
model_noising_4 = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-noising_4") # denoise

# Load Feature Extractor and Tokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

C:\Users\Poco\anaconda3\envs\hty\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Poco\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

# 0. Load TEST Dataset and Suppress Tokens List

In [3]:
from datasets import load_dataset

from transformers import WhisperFeatureExtractor, WhisperTokenizer

from datasets import Audio

# 데이터셋을 학습에 알맞게 준비해보자.
def prepare_dataset(batch):
    # 데이터셋(batch)에서 오디오 정보를 추출한다. (Resampling 포함: 48kHz -> 16kHz)
    audio = batch["audio"]

    # Log-melspectrogram feature를 batch 데이터 정보에 추가한다.
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # text를 토큰화한 id들을 batch 데이터 정보에 추가한다.
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [4]:
dataset = load_dataset("audiofolder", data_dir="./Testset")

# 오디오 샘플을 로드할 때 resampling(sampling rate 조정)을 수행한다.
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

prepared_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=None)

Resolving data files:   0%|          | 0/584 [00:00<?, ?it/s]

In [5]:
test_dataset = prepared_dataset['train']
test_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 252
})

In [6]:
dataset_dns = load_dataset("audiofolder", data_dir="./Testset_Denoising")

# 오디오 샘플을 로드할 때 resampling(sampling rate 조정)을 수행한다.
dataset_dns = dataset_dns.cast_column("audio", Audio(sampling_rate=16000))

prepared_dataset_dns = dataset_dns.map(prepare_dataset, remove_columns=dataset_dns.column_names["train"], num_proc=None)

Resolving data files:   0%|          | 0/584 [00:00<?, ?it/s]

In [7]:
test_dataset_with_dns = prepared_dataset_dns['train']
test_dataset_with_dns

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 252
})

In [8]:
"""
suppress_tokens_list_0 = [7727, 16519, 19266, 19728, 19900, 24155, 25745, 30952, 39045, 41756, 41842, 45433]
suppress_tokens_list_1 = suppress_tokens_list_0 + [9895, 9754, 3558, 4246, 23879, 28387, 343, 44158, 12139, 18079, 12843, 18197,
                                                   13898, 12851, 37609, 2406, 24124, 7911, 88, 4303, 28294, 14938, 8494, 25276,
                                                   24733, 4496, 31064, 14062, 15958, 20793, 7933, 45613, 23538, 23247, 40089,
                                                   33117, 1525, 75, 46081, 35023, 25743, 7490, 17344, 53, 31923, 23317, 74, 43451,
                                                   9012, 49258, 30997, 24624, 288, 43, 11205, 22516, 12145, 7773, 27102, 16133, 10154,
                                                   11453, 21423, 10858, 28898, 48, 9657, 2298, 30827, 21830, 12656, 9125, 48156, 24158,
                                                   13384, 46795, 13626, 16919, 27127, 23399, 30693]
suppress_tokens_list_2 = suppress_tokens_list_1 + [9999, 10191, 44557, 1783, 11816, 11803, 49000, 398, 16513, 32, 8227, 10076, 35016, 39385,
                                                   6866, 28890, 31537, 48437, 65, 85, 37589, 26837, 39615, 41789, 31682, 45, 42692, 8465, 47414,
                                                   10389, 6096, 4702, 46476, 36, 14423, 7271, 11309, 508, 40, 26687, 35, 7771, 21126, 41401, 28638,
                                                   76, 18682, 11901, 6074, 30849, 37785, 31104, 34191, 41619, 28868, 32803, 22350, 27816, 33]
suppress_tokens_list_3 = suppress_tokens_list_2 + [9522, 6494, 13083, 20435, 6905, 12566, 64, 48512, 27228, 26034, 27990, 4550, 2506, 13435, 34716,
                                                   41483, 24, 276, 46478, 11169, 6465, 5442, 41171, 43373, 36630, 8858, 79, 6591, 2803, 14707, 4762,
                                                   12568, 2031, 4436, 22016, 9505, 7634, 9562, 10698, 4808, 30879, 12790, 26157, 22373, 42685, 14034,
                                                   26691, 7562, 40481, 24587, 389, 11650, 15238, 15204, 39177, 51, 72, 82, 29139, 1249, 5211, 17822,
                                                   23943, 7551, 5348, 7546, 6071, 5254, 14695, 18485, 68] # 3to5
suppress_tokens_list_4 = suppress_tokens_list_3 + [1294, 40589, 44930, 45061, 36844, 43476, 21, 8423, 35311, 11624, 66, 3282, 367, 32314, 5080, 3405, 
                                                   3182, 9413, 3446, 30567, 13810, 34328, 11849, 23, 38067, 11971, 21256, 624, 67, 344, 23271, 22671, 
                                                   44353, 47787, 5867, 422, 41112, 33820, 8451, 6879, 8652, 32800, 37401, 261, 16408, 10411, 35604, 
                                                   41712, 37163, 7526, 32086, 18731, 7629, 18065, 29819, 24327, 7201, 22046, 36963, 26267, 12330, 
                                                   29387] # 6to10
suppress_tokens_list_5 = suppress_tokens_list_4 + [43084, 45054, 13947, 15286, 10359, 38667, 297, 49566, 3499, 45934, 3165, 39530, 16685, 8132, 28332, 
                                                   46778, 283, 37770, 8074, 6665, 44014, 27144, 14668, 10353, 4289, 37347, 20984, 35346, 17390, 462, 
                                                   37680, 308, 22621, 4341, 39765, 14892, 36195, 39825, 40279, 33064, 11174, 22, 41867, 2009, 277, 
                                                   27169, 44627, 441, 42872, 479, 39567, 6673, 287, 46869, 46650, 19370, 361, 23777, 43369, 49117, 
                                                   591, 18862, 46034, 34057, 36710, 28569, 497, 460, 6976, 44246, 49441] # 11to20
suppress_tokens_list_6 = suppress_tokens_list_5 + [44532, 19340, 38403, 34071, 33600, 25918, 46977, 27436, 29512, 40603, 38909, 21699, 48864, 34067, 
                                                   47629, 44221, 28981, 48150, 45134, 19220, 23108, 47019, 26416, 9414, 1958, 45549, 741, 46370, 43250, 
                                                   47792, 23531, 17, 25233, 40186, 39469, 39870, 41413, 37861, 38700, 33889, 37222, 47783, 44284, 1360, 
                                                   41123, 36329, 29716, 20, 47992, 33067, 24399, 40071, 628, 47353, 49280, 36036, 18112, 16, 5923, 
                                                   35715, 34895, 23630, 47326, 33411, 31627, 40803, 30060, 46808, 48967, 23178, 274, 28627, 9397, 
                                                   26242, 33502, 46673, 0, 43258, 257, 27056, 43445, 28699, 33750, 34092, 48808, 37336, 34659, 
                                                   43797, 4167, 37102, 24037, 47132, 316, 49805, 37138, 39030, 24148, 32303, 42942, 20688, 36683, 
                                                   4022, 38430, 16239, 30668, 38017, 8214, 363, 17914, 290, 41696, 37604, 3279, 22875, 426, 
                                                   28744, 6641, 40037, 3705, 37255, 44771, 16775, 25457, 23473, 4688, 27146, 383, 22050, 9714, 
                                                   49653, 2443, 413, 29142, 4060, 34371, 36791, 47236, 350, 35362, 36498, 37191, 30580, 47455, 
                                                   33805, 19426, 23111, 46560, 3921, 10791, 45410, 49790, 44298, 5853, 33328, 32840, 3552, 46799, 
                                                   45842, 19289, 18, 430, 45388, 45632, 376, 38791, 45319, 48374, 39077, 12580, 46984, 19, 286, 5285, 
                                                   26707, 33055] # 21to50
"""

'\nsuppress_tokens_list_0 = [7727, 16519, 19266, 19728, 19900, 24155, 25745, 30952, 39045, 41756, 41842, 45433]\nsuppress_tokens_list_1 = suppress_tokens_list_0 + [9895, 9754, 3558, 4246, 23879, 28387, 343, 44158, 12139, 18079, 12843, 18197,\n                                                   13898, 12851, 37609, 2406, 24124, 7911, 88, 4303, 28294, 14938, 8494, 25276,\n                                                   24733, 4496, 31064, 14062, 15958, 20793, 7933, 45613, 23538, 23247, 40089,\n                                                   33117, 1525, 75, 46081, 35023, 25743, 7490, 17344, 53, 31923, 23317, 74, 43451,\n                                                   9012, 49258, 30997, 24624, 288, 43, 11205, 22516, 12145, 7773, 27102, 16133, 10154,\n                                                   11453, 21423, 10858, 28898, 48, 9657, 2298, 30827, 21830, 12656, 9125, 48156, 24158,\n                                                   13384, 46795, 13626, 16919, 27127, 23399, 306

# 1. Define Evaluate Function

In [9]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [11]:
import evaluate

metric_wer = evaluate.load("wer")
metric_cer = evaluate.load("cer")

In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate(model, test_dataset, suppress_tokens_list = []):
    model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")
    model.config.suppress_tokens = suppress_tokens_list
    
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",  # change to a repo name of your choice
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        fp16=True,
        evaluation_strategy="steps",
        per_device_eval_batch_size=16,
        predict_with_generate=True,
        generation_max_length=225,
        save_steps=500,
        eval_steps=100,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="cer",
        greater_is_better=False,
        push_to_hub=False, # True
    )

    model.to(device)
    
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    results = trainer.evaluate(test_dataset)
    print(results)
    return results

# 2. DNS TEST

## 2.1. with DNS

In [14]:
result_211 = evaluate(model_baseline, test_dataset_with_dns)

{'eval_loss': 2.362967014312744, 'eval_wer': 42.847754654983575, 'eval_cer': 24.407343592181093, 'eval_runtime': 104.8945, 'eval_samples_per_second': 2.402, 'eval_steps_per_second': 0.153}


In [15]:
result_212 = evaluate(model_noising, test_dataset_with_dns)

{'eval_loss': 0.42495274543762207, 'eval_wer': 27.38225629791895, 'eval_cer': 12.482918424336047, 'eval_runtime': 98.336, 'eval_samples_per_second': 2.563, 'eval_steps_per_second': 0.163}


In [16]:
result_213 = evaluate(model_denoising, test_dataset_with_dns)

{'eval_loss': 0.4777175784111023, 'eval_wer': 29.98904709748083, 'eval_cer': 13.46919374962866, 'eval_runtime': 97.9651, 'eval_samples_per_second': 2.572, 'eval_steps_per_second': 0.163}


In [17]:
result_214 = evaluate(model_noising_denoising, test_dataset_with_dns)

{'eval_loss': 0.40446150302886963, 'eval_wer': 30.492880613362537, 'eval_cer': 15.21002911294635, 'eval_runtime': 102.2475, 'eval_samples_per_second': 2.465, 'eval_steps_per_second': 0.156}


In [22]:
result_215 = evaluate(model_noising_denoising_2, test_dataset_with_dns)

{'eval_loss': 0.4410058856010437, 'eval_wer': 27.64512595837897, 'eval_cer': 12.981997504604598, 'eval_runtime': 103.126, 'eval_samples_per_second': 2.444, 'eval_steps_per_second': 0.155}


In [15]:
result_216 = evaluate(model_noising_denoising_all, test_dataset_with_dns)

{'eval_loss': 0.35818180441856384, 'eval_wer': 24.03066812705367, 'eval_cer': 10.56384053235102, 'eval_runtime': 107.9817, 'eval_samples_per_second': 2.334, 'eval_steps_per_second': 0.148}


In [16]:
result_217 = evaluate(model_noising_denoising_all_pure, test_dataset_with_dns)

{'eval_loss': 0.36749109625816345, 'eval_wer': 25.21358159912377, 'eval_cer': 11.045095359752837, 'eval_runtime': 103.352, 'eval_samples_per_second': 2.438, 'eval_steps_per_second': 0.155}


In [14]:
result_218 = evaluate(model_noising_4, test_dataset_with_dns)

{'eval_loss': 0.3655948340892792, 'eval_wer': 24.60021905805038, 'eval_cer': 10.932208424930188, 'eval_runtime': 114.0992, 'eval_samples_per_second': 2.209, 'eval_steps_per_second': 0.14}


## 2.2. without DNS

In [18]:
result_221 = evaluate(model_baseline, test_dataset)

{'eval_loss': 2.3337018489837646, 'eval_wer': 41.9277108433735, 'eval_cer': 23.623076466044797, 'eval_runtime': 104.5523, 'eval_samples_per_second': 2.41, 'eval_steps_per_second': 0.153}


In [19]:
result_222 = evaluate(model_noising, test_dataset)

{'eval_loss': 0.3785727322101593, 'eval_wer': 25.10405257393209, 'eval_cer': 11.045095359752837, 'eval_runtime': 98.9665, 'eval_samples_per_second': 2.546, 'eval_steps_per_second': 0.162}


In [20]:
result_223 = evaluate(model_denoising, test_dataset)

{'eval_loss': 0.5208581686019897, 'eval_wer': 39.386637458926614, 'eval_cer': 21.139563899946527, 'eval_runtime': 101.0127, 'eval_samples_per_second': 2.495, 'eval_steps_per_second': 0.158}


In [21]:
result_224 = evaluate(model_noising_denoising, test_dataset)

{'eval_loss': 0.4233647882938385, 'eval_wer': 27.00985761226725, 'eval_cer': 12.263085972312995, 'eval_runtime': 98.631, 'eval_samples_per_second': 2.555, 'eval_steps_per_second': 0.162}


In [23]:
result_225 = evaluate(model_noising_denoising_2, test_dataset)

{'eval_loss': 0.4643176794052124, 'eval_wer': 30.602409638554217, 'eval_cer': 16.338898461172835, 'eval_runtime': 104.6869, 'eval_samples_per_second': 2.407, 'eval_steps_per_second': 0.153}


In [17]:
result_226 = evaluate(model_noising_denoising_all, test_dataset)

{'eval_loss': 0.4188573658466339, 'eval_wer': 30.58050383351588, 'eval_cer': 15.55463133503654, 'eval_runtime': 105.0639, 'eval_samples_per_second': 2.399, 'eval_steps_per_second': 0.152}


In [18]:
result_227 = evaluate(model_noising_denoising_all_pure, test_dataset)

{'eval_loss': 0.4330815374851227, 'eval_wer': 36.03504928806134, 'eval_cer': 18.34115619986929, 'eval_runtime': 104.5615, 'eval_samples_per_second': 2.41, 'eval_steps_per_second': 0.153}


In [26]:
result_228 = evaluate(model_noising_2, test_dataset)

{'eval_loss': 0.377771258354187, 'eval_wer': 25.345016429353777, 'eval_cer': 11.960073673578515, 'eval_runtime': 84.707, 'eval_samples_per_second': 2.975, 'eval_steps_per_second': 0.189}


In [14]:
result_229 = evaluate(model_noising_3, test_dataset)

{'eval_loss': 0.37223950028419495, 'eval_wer': 24.731653888280395, 'eval_cer': 11.056978194997326, 'eval_runtime': 58.4692, 'eval_samples_per_second': 4.31, 'eval_steps_per_second': 0.274}


## 2.3. Suppress tokens

In [30]:
result_230 = evaluate(model_noising, test_dataset, suppress_tokens_list_0)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 132.7403, 'eval_samples_per_second': 2.486, 'eval_steps_per_second': 0.158}


In [31]:
result_231 = evaluate(model_noising, test_dataset, suppress_tokens_list_1)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 135.3671, 'eval_samples_per_second': 2.438, 'eval_steps_per_second': 0.155}


In [32]:
result_232 = evaluate(model_noising, test_dataset, suppress_tokens_list_2)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 132.8588, 'eval_samples_per_second': 2.484, 'eval_steps_per_second': 0.158}


In [33]:
result_233 = evaluate(model_noising, test_dataset, suppress_tokens_list_3)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 134.3551, 'eval_samples_per_second': 2.456, 'eval_steps_per_second': 0.156}


In [39]:
result_234 = evaluate(model_noising, test_dataset, suppress_tokens_list_4)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 135.1723, 'eval_samples_per_second': 2.441, 'eval_steps_per_second': 0.155}


In [40]:
result_235 = evaluate(model_noising, test_dataset, suppress_tokens_list_5)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 137.3349, 'eval_samples_per_second': 2.403, 'eval_steps_per_second': 0.153}


In [38]:
result_236 = evaluate(model_noising, test_dataset, suppress_tokens_list_6)

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 136.1154, 'eval_samples_per_second': 2.424, 'eval_steps_per_second': 0.154}


In [44]:
result_237 = evaluate(model_noising, test_dataset, [i for i in range(50000)])

{'eval_loss': 0.3332138657569885, 'eval_wer': 22.227772227772228, 'eval_cer': 9.664453777717563, 'eval_runtime': 139.1267, 'eval_samples_per_second': 2.372, 'eval_steps_per_second': 0.151}


- Suppress Tokens는 추론 시에는 적용이 안 되는 듯

# 3. GPT TEST

## 태연 프롬프트

In [25]:
def STT(audio_path, model):
    y, s = librosa.load(audio_path, sr=16000) # sampling rate 변경 (44100 -> 16000)
    input_features = processor(y, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    predicted = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    print(f"STT result: {predicted}")
    return predicted

def ChatGPT(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "주어진 문장에서 틀린 글자와 단어를 맥락에 맞게 모두 고치세요."},
            {"role": "system", "content": "구두점(마침표, 쉼표, 느낌표, 물음표)이 표기되어 있지 않으면 구두점을 추가로 표기하세요."},
            {"role": "system", "content": "단, 고친 문장만을 출력하세요. 다른 어떠한 부가적인 문장 추가하지 마세요."},
            {"role": "user", "content": text},
        ],
        temperature=0.1, # temperature 설정 (높은 값: 출력이 무작위로 생성, 낮은 값: 출력이 일관성 있게 생성)
    )

    output_text = response["choices"][0]["message"]["content"]
    print(f"ChatGPT result: {output_text}")
    return output_text

In [26]:
import json
import time

def sub(audio_path, model):
    start = time.time()
    stt_text = STT(audio_path, model)
    print(time.time() - start)
    gpt_text = ChatGPT(stt_text)
    print(time.time() - start)
    
    # Answer
    json_path = audio_path.replace('audio', 'label')
    json_path = json_path.replace('wav', 'json')
    with open(json_path, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
        answer = data["annotations"]["note"]
        print(f"Answer: {answer}")
    return stt_text, gpt_text, answer

In [30]:
def main(audio_path_list, model):
    model.to('cpu')
    stt_text_list = []
    gpt_text_list = []
    answer_list = []
    for audio_path in audio_path_list:
        stt_text, gpt_text, answer = sub(audio_path, model.to('cpu'))
        stt_text_list.append(stt_text)
        gpt_text_list.append(gpt_text)
        answer_list.append(answer)
    return stt_text_list, gpt_text_list, answer_list

In [28]:
import os
file_list = []
for root, dirs, files in os.walk(".\\GPT_Testset\\audio"):
    for filename in files:
        filename = os.path.join(root, filename)
        file_list.append(filename)
file_list # 35개

['.\\GPT_Testset\\audio\\01_01_000080_210729_SN_3.wav',
 '.\\GPT_Testset\\audio\\01_01_000596_210809_SD_7.wav',
 '.\\GPT_Testset\\audio\\01_01_010456_211012_SD_4.wav',
 '.\\GPT_Testset\\audio\\02_01_044620_211105_SD_6.wav',
 '.\\GPT_Testset\\audio\\02_01_052982_211118_SD_0.wav',
 '.\\GPT_Testset\\audio\\02_01_063169_211122_SD_7.wav',
 '.\\GPT_Testset\\audio\\02_03_046293_211106_SD_4.wav',
 '.\\GPT_Testset\\audio\\03_01_000857_210812_SN_19.wav',
 '.\\GPT_Testset\\audio\\03_01_027274_211025_SN_6.wav',
 '.\\GPT_Testset\\audio\\03_01_039953_211103_SN_15.wav',
 '.\\GPT_Testset\\audio\\04_01_000730_210806_SD_0.wav',
 '.\\GPT_Testset\\audio\\04_01_025887_211023_SD_18.wav',
 '.\\GPT_Testset\\audio\\04_01_044141_211107_SN_9.wav',
 '.\\GPT_Testset\\audio\\05_01_038549_211029_SD_27.wav',
 '.\\GPT_Testset\\audio\\05_01_049935_211112_SN_8.wav',
 '.\\GPT_Testset\\audio\\05_03_009408_210923_SD_7.wav',
 '.\\GPT_Testset\\audio\\06_01_000021_210716_SD_6.wav',
 '.\\GPT_Testset\\audio\\06_01_006969_210908

In [29]:
main(file_list[0:1], model_noising_denoising_all)

STT result:  마음으로는 끼우고 싶은데 제가 마음이 약해서 한번 더 오지만 세기가 정말 힘들거든요 그래서 다 하지는 못했지만 이 감고기 많다 그러면서 안 끼우고 싶어요
ChatGPT result: 마음으로는 끼우고 싶은데, 제가 마음이 약해서 한 번 더 오지만 세기가 정말 힘들거든요. 그래서 다 하지는 못했지만 이 감고기 많다고 하면서 안 끼우고 싶어요.
Answer: 마음으로는 키우고 싶은데 제가 마음이 약해서 한 번 정을 주면 떼기가 정말 힘들거든요 그래서 강아지는 귀엽지만 괜히 정주지 말자 이러면서 안 키우고 있어요.


([' 마음으로는 끼우고 싶은데 제가 마음이 약해서 한번 더 오지만 세기가 정말 힘들거든요 그래서 다 하지는 못했지만 이 감고기 많다 그러면서 안 끼우고 싶어요'],
 ['마음으로는 끼우고 싶은데, 제가 마음이 약해서 한 번 더 오지만 세기가 정말 힘들거든요. 그래서 다 하지는 못했지만 이 감고기 많다고 하면서 안 끼우고 싶어요.'],
 ['마음으로는 키우고 싶은데 제가 마음이 약해서 한 번 정을 주면 떼기가 정말 힘들거든요 그래서 강아지는 귀엽지만 괜히 정주지 말자 이러면서 안 키우고 있어요.'])

In [46]:
S, G, A = main(file_list, model_noising)

STT result:  마음으로는 끼우고 싶은데 제가 마음이 약해서 한번 더 오지만 세기가 정말 힘들거든요 그래서 다 하지는 못했지만 이 감고기 많다 그러면서 안 끼우고 싶어요
ChatGPT result: 마음으로는 끼우고 싶은데, 제가 마음이 약해서 한 번 더 오지만 세기가 정말 힘들거든요. 그래서 다 하지는 못했지만 이 감고기 많다고 하면서 안 끼우고 싶어요.
Answer: 마음으로는 키우고 싶은데 제가 마음이 약해서 한 번 정을 주면 떼기가 정말 힘들거든요 그래서 강아지는 귀엽지만 괜히 정주지 말자 이러면서 안 키우고 있어요.
STT result:  계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
ChatGPT result: 계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
Answer: 계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
STT result:  아 그러시구나. 그렇군요. 어디 한번 말씀을 더 해보시죠.
ChatGPT result: 아, 그러시구나. 그렇군요. 어디 한 번 말씀을 더 해보시죠.
Answer: 그러시구나. 그렇군요. 어디 한 번 말씀을 더 해보시죠.
STT result:  이제 깨달았네요. 저희가 너무 뛰어난 예술가형이라서 이렇게 공통점이 생기는 것 같습니다. 그러면 또 물어보고 싶은 게 있는데요. 여행을 마치고 돌아갔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요. 1번 거기 완전 대박이야.
ChatGPT result: 이제 깨달았네요. 저희가 너무 뛰어난 예술가형이라서 이렇게 공통점이 생기는 것 같습니다. 그러면 또 물어보고 싶은 게 있는데요. 여행을 마치고 돌아갔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요. 1번, 거기 완전 대박이야.
Answer: 이제 깨달았네요 저희가 너무 뛰어난 예술가 형이라서 이렇게 공통점이 생기는 것 같습니다 그러면 또 물어보고 싶은 게 있는데요 여행을 마치고 돌아왔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요 1

In [47]:
import nlptutti

stt_wer, stt_cer, gpt_wer, gpt_cer = 0, 0, 0, 0
for s, g, a in zip(S, G, A):
    stt_wer += nlptutti.get_wer(s, a)['wer'] / len(S)
    stt_cer += nlptutti.get_cer(s, a)['cer'] / len(S)
    gpt_wer += nlptutti.get_wer(g, a)['wer'] / len(S)
    gpt_cer += nlptutti.get_cer(g, a)['cer'] / len(S)

print(f"STT | WER: {stt_wer} | CER: {stt_cer}")
print(f"GPT | WER: {gpt_wer} | CER: {gpt_cer}")

STT | WER: 0.3153756927754966 | CER: 0.18003260206015256
GPT | WER: 0.32715830285003217 | CER: 0.21577683904486566


## 세정님 프롬프트

In [14]:
def ChatGPT(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 STT가 출력한 문장을 입력으로 받습니다."},
            {"role": "system", "content": "당신은 원본 음성이 무엇이었을지 적절히 추측하며 맥락에 맞게 문장을 수정해야 합니다."},
            {"role": "system", "content": "당신은 입력에서 틀린 단어만 수정합니다."},
            {"role": "system", "content": "당신은 입력을 최소한으로 수정합니다."},
            {"role": "system", "content": "당신은 입력 문장과 비슷하게 출력해야 합니다."},
            {"role": "system", "content": "입력 어투와 출력 어투는 같아야 합니다."},
            {"role": "system", "content": "당신은 올바르게 수정한 문장만을 출력합니다."},
            {"role": "user", "content": "미안한데 네가 무슨 말을 하는지 모르겠어 그리고 네가 학교 생활을 어떻게 해야 될지 왜 나한테 물어봐 그런 건 네이버 지식인회다 물어보거든요 여기 그리고 기숙자만인데 들리지 마 해당 기소리보다 목소리가 더 커 너 무슨 소문이 났다고 그러는데"},
            {"role": "assistant", "content": "미안한데 네가 무슨 말 하는지 잘 모르겠어. 그리고 네가 학교 생활을 어떻게 해야 될지 왜 나한테 물어봐. 그런 건 네이버 지식인에다 물어보든지. 여기 그리고 기숙사 방인데. 조용히 좀 해. 세탁기 소리보다. 네 목소리가 더 커. 너 무슨 소문이 났다고 그러는데?"},
            {"role": "user", "content": "아 그런 격땐 쓴맛이 당신을 만족시킨 맛인 것 같은데 어떻게 해드려야 제가 될까요? 그 고기 좀 넣어서 그럼 한 맛을 중화시키되? 너무 없지는 않게 해서 아메리카만 만들어드리면 만들 수 있을까요?"},
            {"role": "assistant", "content": "그럼 결국엔 쓴맛이 당신을 만족시키는 맛인 것 같은데 어떻게 해드려야 제가 될까요? 그 두유를 넣어서 그럼. 쓴맛을 중화시키되 너무 없지는 않게 해서 아메리카노를 만들어드리면 만족하실까요?"},
            {"role": "user", "content": "빌라촌이도 굴목들로 만들어져있었기 때문에 그런 조깅문화가 사실 황성이 잘 안되고 있다라고 생각이 들 수가 없더라고요. 사실 그냥 조깅문화가 한국에서 하는 사람들은 방강 쪽이나 대천 쪽으로 서울에서 있는 불광청, 중랑청"},
            {"role": "assistant", "content": "빌라촌이나 이런 거에서 골목들로 많이 이루어져 있기 때문에 그런 조깅 문화가 사실 형성이 잘 안 되어 있다라고 생각이 들 수밖에 없더라고요. 사실 그냥 조을 한국에서 한 사람들은 한강 쪽이나 이제 개천 쪽으로 이제 서울에서는 불광천이나 중랑천"},
            {"role": "user", "content": "그 느낌처럼 연금 복원을 받게 되면 매달 나오는 구매를 어디다가 쓸까? 교육하면서 기다리는 재미가 있을 것 같아요 그러면서 세월구스 정말 행복하게 흘러가고 좋을 것 같은데"},
            {"role": "assistant", "content": "그런 그 느낌처럼 연금 복권을 받게 되면 매달 나오는 금액을 어디다가 쓸까 계획하면서 기다리는 재미가 있을 것 같아. 그러면서 세월도 정말 행복하게 흘러가고 좋을 것 같은데."},
            {"role": "user", "content": "수지씨는 왜 연애가 끝나고 이별을 하게 되고 난 다음에 혹시 그 사람이 보고 싶다거나 아니면 우연적으로 만났다거나 혹시 그런 적 있나요?"},
            {"role": "assistant", "content": "수지 씨는 왜 연애가 끝나고 이별을 하게 되고 난 다음에 혹시 그 사람이 보고 싶다거나 아니면 우연적으로 만났다거나. 혹시 그런 적 있나요?"},
            {"role": "user", "content": "원데이 클래스 그러니까 어떤 것이든 간에 어떤 종목이 된다면 딱 하나의 한에서 하루 동안에 점수를 넣을 수 있다면 너는 어떤 걸 배워보고 싶어? 이게 숙달 정도는 보통 이상이야 완전히 프로필심을 하는 게 아니고 보통 이상의 점수를 넣을 수 있어 너는 어떤 걸 배워보고 싶어?"},
            {"role": "assistant", "content": "원데이 클래스 그러니까 어떤 것이든 간에 어떤 종목이든 간에 딱 하나에 한해서 하루 동안에 전부 배울 수 있다면 너는 어떤 걸 배워보고 싶어? 이게 숙달 정도는 보통 이상이야. 완전히 프로페셔널한 게 아니고 보통 이상 정도. 너는 어떤 걸 배워보고 싶어?"},
            {"role": "system", "content": "어떤 로망이 있으셨죠?"},
            {"role": "system", "content": "어떤 로망이 있으셨죠?"},
            {"role": "system", "content": "우리 집은 사실 내가 보섬이 되기 전까지는 그렇게 화목한 편이라고 하지는 못했어 나를 제외한 우리 집은 화목했었던 것 같은데 내가 그 사이에 들어가면 어딘가 불편한 느낌이었어 내가 사춘기를 조금 유별나게 겪었었던 것 같아"},
            {"role": "system", "content": "우리 집은 사실 내가 고3이 되기 전까지는 그렇게 화목한 편이라고 하지는 못했어. 나를 제외한 우리 집은 화목했었던 것 같은데. 내가 그 사이에 들어가면 어딘가 불편한 느낌이었어. 내가 사춘기를 조금 유별하게 겪었었던 것 같아."},
            {"role": "user", "content": text},
        ],
        temperature=0.1, # temperature 설정 (높은 값: 출력이 무작위로 생성, 낮은 값: 출력이 일관성 있게 생성)
        top_p=0.1, 
        request_timeout=15
    )

    output_text = response["choices"][0]["message"]["content"]
    print(f"ChatGPT result: {output_text}")
    return output_text

In [41]:
S, G, A = main(file_list, model_noising)

STT result:  마음으로는 끼우고 싶은데 제가 마음이 약해서 한번 더 오지만 세기가 정말 힘들거든요 그래서 다 하지는 못했지만 이 감고기 많다 그러면서 안 끼우고 싶어요
ChatGPT result: 마음으로는 끼우고 싶은데 제가 마음이 약해서 한 번 더 오지만 세기가 정말 힘들거든요. 그래서 다 하지는 못했지만 이 감고기 많다 그러면서 안 끼우고 싶어요.
Answer: 마음으로는 키우고 싶은데 제가 마음이 약해서 한 번 정을 주면 떼기가 정말 힘들거든요 그래서 강아지는 귀엽지만 괜히 정주지 말자 이러면서 안 키우고 있어요.
STT result:  계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
ChatGPT result: 계획을 세우지 않거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
Answer: 계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
STT result:  아 그러시구나. 그렇군요. 어디 한번 말씀을 더 해보시죠.
ChatGPT result: 아 그러시구나. 그렇군요. 어디 한번 말씀을 더 해보시죠.
Answer: 그러시구나. 그렇군요. 어디 한 번 말씀을 더 해보시죠.
STT result:  이제 깨달았네요. 저희가 너무 뛰어난 예술가형이라서 이렇게 공통점이 생기는 것 같습니다. 그러면 또 물어보고 싶은 게 있는데요. 여행을 마치고 돌아갔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요. 1번 거기 완전 대박이야.
ChatGPT result: 이제 깨달았네요. 우리가 너무 뛰어난 예술가형이라서 이렇게 공통점이 생기는 것 같습니다. 그러면 또 물어보고 싶은 게 있는데요. 여행을 마치고 돌아갔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요. 1번 거기 완전 대박이야.
Answer: 이제 깨달았네요 저희가 너무 뛰어난 예술가 형이라서 이렇게 공통점이 생기는 것 같습니다 그러면 또 물어보고 싶은 게 있는데요 여행을 마치고 돌아왔을 때 사람들에게 어떻게 얘기할 것인가에 대한 거예요 1 번 

In [42]:
import nlptutti

stt_wer, stt_cer, gpt_wer, gpt_cer = 0, 0, 0, 0
for s, g, a in zip(S, G, A):
    stt_wer += nlptutti.get_wer(s, a)['wer'] / len(S)
    stt_cer += nlptutti.get_cer(s, a)['cer'] / len(S)
    gpt_wer += nlptutti.get_wer(g, a)['wer'] / len(S)
    gpt_cer += nlptutti.get_cer(g, a)['cer'] / len(S)

print(f"STT | WER: {stt_wer} | CER: {stt_cer}")
print(f"GPT | WER: {gpt_wer} | CER: {gpt_cer}")

STT | WER: 0.3153756927754966 | CER: 0.18003260206015256
GPT | WER: 0.41705468725487915 | CER: 0.2909993757448092


# 3-1. GPT TEST (REVISE)

In [30]:
predictions = []
labels = []
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    predictions.extend(pred_str)
    labels.extend(label_str)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [33]:
result_216 = evaluate(model_noising_denoising_all, test_dataset_with_dns)

{'eval_loss': 0.35818180441856384, 'eval_wer': 24.03066812705367, 'eval_cer': 10.56384053235102, 'eval_runtime': 164.6397, 'eval_samples_per_second': 1.531, 'eval_steps_per_second': 0.097}


In [34]:
predictions

['그건 모르죠.',
 '1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 네 개 정도는 이기 될 수도 있을 것 같아. 배송류가 얼만지에 따라 다르겠지.',
 '그래 지금 우리 집사람도 빨리 오고 싶지 나한테 문자로 빨리 갈게 탁 들고 가 이렇게 문자까지 보냈어 내가 그거 자네한테 이야기해줬지',
 '그래 아까 나름대로 작전을 잘 짰던 것 같아 만두부터 먹고 그다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 되지만 광고 속에 있는 먹는 선전은 별로 맞았지 않지',
 '아니요. 저희는 지금 베란다가 좀 좁은 편이라서 빨래에 넣을 공간이 별로 없거든요. 그래서 지금 가끔씩은 거실에다가 빨래를 넣이기도 하고요. 좀 자잘한 것 같은 경우는 한 번에 몰아서 베란다에 말리기도 하고 있어요.',
 '현미야. 너네 아파트는 분리수거 며칠 날 하니?',
 '쯧쯧한 거 괜찮을까?',
 '왜? 나는 꽃돌을 변신하는 거 가지고 싶은데.',
 '계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.',
 '맞아. 많은 사람들이 부동산 불폐라면서 부동산 매주에 참여하고 있어. 특히 이웅 성동 세대가 눈부르를 안 가리고 참여하고 있어서 걱정이야.',
 '그렇게 하면 곰팡이도 안 쓰고 물 때도 안 끼고 좋은데 관리하기가 정말 정말 힘들지.',
 '극당이라는 것도 그것에서 좋았어. 아마 그런 행위를 하는 것은 그런 사업자는 당신 또한 몸이 아픈 사람이 마음을 잡아주는 그런 역할을 한다고 생각해. 그렇지만 사입이 종교와',
 '이번 주에는 이제 체크 세트 아이템들은 많이 생각할 것 같긴 해요. 체크 자체 매력적인 게 가장 좋은 포기조합은 뭐라고 생각하시나요.',
 '자연스러운 발톱만큼 발톱이 잘한 거야. 근데 나만 그런 게 아니고 대부분의 여자들이 샌들을 신는 여름에는 열심히 헤디큐어를 하지만',
 '일주일에 두 번 그러면 그렇게 많은 시간이 없네요.',
 '그렇죠 저는 아

In [35]:
labels

['그건 모르죠',
 '어 원 플러스 원이라고 해도 지금 행사니까 원 플러스 원인 거지 만약에 따로 사게 되면은 한 네 개 정도는 이득 될 수도 있을 것 같아 배송료가 얼마인지에 따라 다르겠지만.',
 '그래 지금 우리 집사람도 빨리 오고 싶지 나한테 문자로 빨리 갈게 차 끌고 가 이렇게 문자까지 보냈어 내가 그거 자네한테 이야기해 줬지',
 '그래 아까 나름대로 작전을 잘 짰던 것 같아 만두부터 먹고 그다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아 먹방은 솔직히 그 먹는 과정에 나와서 먹고 싶은 마음이 들지만 광고 속에 있는 먹는 선전은 별로 와닿지 않지',
 '아니요. 저희는 지금 베란다가. 좀 좁은 편이라서. 빨래 널 공간이 별로 없거든요. 그래서 지금 가끔씩은 거실에다가 빨래를 널기도 하고요. 좀 자잘한 것 같은 경우는 한 번에 몰아서 베란다에 말리기도 하고 있어요.',
 '현미야. 너네 아파트는 분리수거 며칠 날 하니?',
 '그 시간도 괜찮을까.',
 '왜 난 모코코로 변신하는 거 가지고 싶은데.',
 '계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.',
 '맞아. 많은 사람들이 부동산 불패라면서 부동산 매수에 참여하고 있어. 특히 2 0 3 0 세대가 물불을 안 가리고 참여하고 있어서 걱정이야.',
 '그렇게 하면 곰팡이도 안 슬고 물때도 안 끼고 좋은데 관리하기가 정말 정말 힘들지.',
 '굿당이라는 것도 그곳에서 보았어. 아마 그런 행위를 하는 것은 그런 사업가나 정치인, 또한 몸이 아픈 사람의 마음을 잡아주는 그런 역할을 한다고 생각돼. 그렇지만 사이비 종교와',
 '이번 연도에는 이제 체크 패턴 아이템들을 많이 선보였을 텐데 체크 자체의 매력을 보여주기에 가장 좋은 코디 조합은 뭐가 있겠을까',
 '자연스러운 발톱만큼 발톱이 자란 거야 근데 나만 그런 게 아니고 대부분의 여자들이 샌들을 신는 여름에는 열심히 페디큐어를 하지만',
 '일주일에 두 번 그러면 그렇게 많은 시간이 아니네요.',
 '그렇죠. 저는 아

In [36]:
import time

chatgpt = []
for text in predictions:
    while(True):
        try:
            gpt_text = ChatGPT(text)
            chatgpt.append(gpt_text)
            break
        except Exception as e:
            if e:
                print(e)
                print('Timeout error, retrying...')
                time.sleep(5)
            else:
                raise e

ChatGPT result: 그건 모르죠.
ChatGPT result: 1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 네 개 정도는 이기 될 수도 있을 것 같아. 배송료가 얼마인지에 따라 다르겠지.
ChatGPT result: 그래, 지금 우리 집 사람도 빨리 오고 싶지. 나한테 문자로 빨리 갈게. 탁 들고 가. 이렇게 문자까지 보냈어. 내가 그거 자네한테 이야기해줬지.
ChatGPT result: 그래, 아까 나름대로 작전을 잘 짰던 것 같아. 만두부터 먹고, 그 다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아. 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 되지만, 광고 속에 있는 먹는 선전은 별로 맞지 않았지.
ChatGPT result: 아니요. 저희는 지금 베란다가 좀 좁은 편이라서 빨래에 넣을 공간이 별로 없거든요. 그래서 지금 가끔씩은 거실에다가 빨래를 넣기도 하고요. 좀 자잘한 것 같은 경우는 한 번에 몰아서 베란다에 말리기도 하고 있어요.
ChatGPT result: 현미야, 너희 아파트는 분리수거 며칠에 하니?
ChatGPT result: 쯧쯧, 한 거 괜찮을까요?
ChatGPT result: 왜? 나는 꽃돌을 변신하는 걸 가지고 싶은데.
ChatGPT result: 계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
ChatGPT result: 맞아. 많은 사람들이 부동산 버블이라면서 부동산 매매에 참여하고 있어. 특히 이웅 성동 세대가 눈부심을 안 가리고 참여하고 있어서 걱정이야.
ChatGPT result: 그렇게 하면 곰팡이도 안 생기고 물 때도 안 끼고 좋은데, 관리하기가 정말 정말 힘들지요?
ChatGPT result: 극단이라는 것도 그것에서 좋았어요. 아마 그런 행위를 하는 것은 그런 사업자는 당신 또한 몸이 아픈 사람이 마음을 잡아주는 그런 역할을 한다고 생각해요. 그렇지만 사입이 종교와 관련된다면, 그것은 더욱 의미있는 일이 될 수 있을 거예요.

In [20]:
cer = 100 * metric_cer.compute(predictions=predictions, references=labels)

In [21]:
cer

10.56384053235102

In [22]:
# 단어, temp=0.1
cer = 100 * metric_cer.compute(predictions=chatgpt, references=labels)

In [23]:
cer

17.776721525756045

In [37]:
# 맥락, 글자, 단어, temp=0.1
cer = 100 * metric_cer.compute(predictions=chatgpt, references=labels)
cer

17.741073020022576

In [27]:
for i, j, k in zip(predictions, chatgpt, labels):
    print('Prediction:', i)
    print('ChatGPT:', j)
    print('Labels:', k)

Prediction: 그건 모르죠.
ChatGPT: "그건 모르죠."
Labels: 그건 모르죠
Prediction: 1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 네 개 정도는 이기 될 수도 있을 것 같아. 배송류가 얼만지에 따라 다르겠지.
ChatGPT: 1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 네 개 정도는 이기 될 수도 있을 것 같아. 배송료가 얼마인지에 따라 다르겠지.
Labels: 어 원 플러스 원이라고 해도 지금 행사니까 원 플러스 원인 거지 만약에 따로 사게 되면은 한 네 개 정도는 이득 될 수도 있을 것 같아 배송료가 얼마인지에 따라 다르겠지만.
Prediction: 그래 지금 우리 집사람도 빨리 오고 싶지 나한테 문자로 빨리 갈게 탁 들고 가 이렇게 문자까지 보냈어 내가 그거 자네한테 이야기해줬지
ChatGPT: 그래, 지금 우리 집 사람도 빨리 오고 싶지. 나한테 문자로 빨리 갈게. 탁 들고 가! 이렇게 문자까지 보냈어. 내가 그거 자네한테 이야기해줬지.
Labels: 그래 지금 우리 집사람도 빨리 오고 싶지 나한테 문자로 빨리 갈게 차 끌고 가 이렇게 문자까지 보냈어 내가 그거 자네한테 이야기해 줬지
Prediction: 그래 아까 나름대로 작전을 잘 짰던 것 같아 만두부터 먹고 그다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 되지만 광고 속에 있는 먹는 선전은 별로 맞았지 않지
ChatGPT: 그래, 아까 나름대로 작전을 잘 짰던 것 같아. 만두부터 먹고, 그 다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아. 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 되지만, 광고 속에 있는 먹는 선전은 별로 맞지 않았지.
Labels: 그래 아까 나름대로 작전을 잘 짰던 것 같아 만두부터 먹고 그다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아 먹방은 솔직히 그 먹

In [17]:
# 세정 프롬프트
import time

chatgpt = []
for text in predictions:
    while(True):
        try:
            gpt_text = ChatGPT(text)
            chatgpt.append(gpt_text)
            break
        except Exception as e:
            if e:
                print(e)
                print('Timeout error, retrying...')
                time.sleep(5)
            else:
                raise e

ChatGPT result: 그건 모르시겠다니까요.
ChatGPT result: 1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 4 개 정도는 이득될 수도 있을 것 같아. 배송료가 얼만지에 따라 다르겠지.
ChatGPT result: 그래, 지금 우리 집 사람도 빨리 오고 싶어. 나한테 문자로 빨리 알려줄게. 찾아들어가고 가. 이렇게 문자까지 보내줘서 내가 그거 자네한테 이야기해줬지.
ChatGPT result: 그래, 아까 나름대로 작전을 잘 짰던 것 같아. 만두부터 먹고 그 다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아. 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 들지만 광고 속에 있는 먹는 선정은 별로 맞지 않았어.
ChatGPT result: 아, 이해했습니다. 저희는 지금 베란다가 좀 좁은 편이라서 빨래를 넓게 말리기에는 공간이 부족한 거군요. 그래서 가끔씩은 거실에다가 빨래를 넓기도 하고, 자잘한 빨래는 한 번에 몰아서 베란다에 말리기도 하고 계시는군요.
ChatGPT result: 현미야, 너희 아파트는 분리수거 며칠에 하는 거야?
ChatGPT result: 글쎄다고? 괜찮을까요?
ChatGPT result: 왜, 나는 이제 꽃불을 변신하는 능력을 갖고 싶구나.
ChatGPT result: 계획을 세우지 않거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
ChatGPT result: 맞아. 많은 사람들이 부동산 버블이라면서 부동산 투자에 참여하고 있어. 특히 20~30세대가 눈치 없이 참여하고 있어서 걱정이야.
ChatGPT result: 그렇게 하면 곰팡이도 안 생기고 물 때도 안 끼고 정말 좋은데, 관리하기가 정말 힘들지.
ChatGPT result: 극단이라는 것도 그것에서 좋았어. 아마 그런 행위를 하는 것은 그런 사업장은 정치인 또한 몸이 아픈 사람이 마음을 잡아주는 그런 역할을 한다고 생각해. 그렇지만 사입이 좀 더...
ChatGPT result: 이번 주에는 이

In [18]:
cer = 100 * metric_cer.compute(predictions=chatgpt, references=labels)

In [19]:
cer

32.588675658012

In [22]:
# 태연 프롬프트 :: '단어'를 고치는 명령을 추가함.
import time

chatgpt = []
for text in predictions:
    while(True):
        try:
            gpt_text = ChatGPT(text)
            chatgpt.append(gpt_text)
            break
        except Exception as e:
            if e:
                print(e)
                print('Timeout error, retrying...')
                time.sleep(5)
            else:
                raise e

ChatGPT result: 그건 모르죠?
ChatGPT result: 1 플러스 1이라고 해도 지금 행사니까 1 플러스 1인 거지. 만약에 따로 사게 되면은 한 4 개 정도는 이득될 수도 있을 것 같아. 배성류가 얼만큼인지에 따라 다르겠지.
ChatGPT result: 그래, 지금 우리 집 사람도 빨리 오고 싶지. 나한테 문자로 빨리 갈게. 찾아들고 가. 이렇게 문자까지 보내줘서, 내가 그거 자네한테 이야기해줬지.
ChatGPT result: 그래, 아까 나름대로 작전을 잘 짰던 것 같아. 만두부터 먹고, 그 다음에 맛있는 탕수육까지 먹었으니 작전을 잘 짠 것 같아. 먹방은 솔직히 그 먹는 과정이 나와서 먹고 싶은 마음이 되지만, 광고 속에 있는 먹는 선정은 별로 맞지 않았지.
ChatGPT result: 아니요, 저희는 지금 베란다가 좀 좁은 편이라서 빨래 넓은 공간이 별로 없거든요. 그래서 가끔씩은 거실에다가 빨래를 넓기도 하고요. 좀 자잘한 것 같은 경우는 한 번에 몰아서 베란다에 말리기도 하고 있어요.
ChatGPT result: 현미야, 너희 아파트는 분리수거 며칠에 하는 거야?
ChatGPT result: 글쎄요. 괜찮을까요?
ChatGPT result: 왜? 나는 이제 꽃불을 변신하는 것을 변신하고 싶구나.
ChatGPT result: 계획을 안 세우거나 메모를 안 하면 정말 불안해서 견딜 수가 없어요.
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Timeout error, retrying...
ChatGPT result: 맞아, 많은 사람들이 부동산 버블이라면서 부동산 매매에 참여하고 있어. 특히 20~30대가 눈에 띄지 않고 참여하고 있어서 걱정이야.
ChatGPT result: 그렇게 하면 곰팡이도 안 생기고 물 때도 안 끼고 좋은데, 관리하기가 정말 정말 힘들지요?
ChatGPT result:

In [23]:
cer = 100 * metric_cer.compute(predictions=chatgpt, references=labels)
cer

18.798645356782128